In [1]:
import h5py
import numpy as np


sem_level = 'coarse' # ['coarse', 'fine']
split = 'test' #['valid', 'test']

data = h5py.File('Exp/3dcompat/com50_coarse/result/best/{}_outputs_2.h5'.format(split))

# test_outputs_50comp

pred_mats = np.array(data['pred_mats']).astype('uint16')
gt_mats = np.array(data['gt_mats']).astype('uint16')

pred_parts = np.array(data['pred_parts']).astype('uint16')
gt_parts = np.array(data['gt_parts']).astype('uint16')

pred_objs = np.array(data['pred_objs']).astype('uint8')
gt_objs = np.array(data['gt_objs']).astype('uint8')

model_ids = np.array(data['model_ids']).astype('S6')[:,0]
style_ids = np.array(data['style_ids']).astype('S6')[:,0]

np.sum((pred_parts)==gt_parts) / (gt_parts.shape[0] * gt_parts.shape[1])

0.004381585576731688

In [2]:
np.unique(pred_objs), np.unique(gt_objs), np.sum(gt_objs==0), gt_objs.shape

(array([ 1,  5,  7,  8, 13, 18, 19, 20, 22, 23, 31, 34, 35, 37, 40, 41],
       dtype=uint8),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41], dtype=uint8),
 70,
 (12560,))

In [3]:
for sid in range(10):
    print(sid, np.sum(style_ids.astype('int')==sid))

0 1256
1 1256
2 1256
3 1256
4 1256
5 1256
6 1256
7 1256
8 1256
9 1256


In [4]:
pred_parts = pred_parts - 1
pred_mats = pred_mats - 1
gt_parts = gt_parts - 1
gt_mats = gt_mats - 1

np.unique(pred_mats), np.unique(gt_mats), np.unique(pred_parts), np.unique(gt_parts)

(array([    0,     1,     2,     3,     5,     6,     7,     8,     9,
           10, 65535], dtype=uint16),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=uint16),
 array([    0,     1,     2,     3,     4,     5,     6,     7,     8,
            9,    10,    11,    13,    14,    15,    16,    17,    18,
           19,    20,    22,    23,    24,    25,    27,    28,    29,
           30,    31,    32,    33,    34,    35,    36,    37,    38,
           39,    41, 65535], dtype=uint16),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42], dtype=uint16))

In [5]:
np.sum((pred_mats)==gt_mats) / (gt_mats.shape[0] * gt_mats.shape[1])

0.11736831179090366

In [6]:
len(model_ids)

12560

In [7]:

data = {'pred_parts': pred_parts, 'gt_parts': gt_parts, 'pred_mats': pred_mats, \
        'gt_mats': gt_mats, 'pred_objs': pred_objs, 'gt_objs': gt_objs, \
       'out_model_ids': model_ids, 'out_style_ids': style_ids
       }

print(pred_parts.shape, gt_parts.shape, pred_mats.shape, gt_mats.shape, pred_objs.shape, gt_objs.shape)


(12560, 2048) (12560, 2048) (12560, 2048) (12560, 2048) (12560,) (12560,)


In [8]:
## if no point_grouping and part_mat_pairs are provided

point_grouping =  -1 * np.ones(pred_parts.shape)
for s in range(len(point_grouping)):
    all_parts = np.unique(pred_parts[s])
    for ii, p in enumerate(all_parts):
        idx = np.where(pred_parts[s]==p)[0]
        point_grouping[s][idx] = ii

part_mat_pairs = -1 * np.ones((point_grouping.shape[0], 100, 2))

for s in range(len(point_grouping)):
    unique_group = np.unique(point_grouping[s])
    
    for ii, g in enumerate(unique_group):
        idx = np.where(point_grouping[s]==g)[0]
        # part grouping
        part_g = pred_parts[s][idx]
        unique_values, value_counts = np.unique(part_g, return_counts=True)
        index_of_most_frequent_value = np.argmax(value_counts)
        vote_part = unique_values[index_of_most_frequent_value]
        
        # material grouping
        mat_g = pred_mats[s][idx]
        unique_values, value_counts = np.unique(mat_g, return_counts=True)
        index_of_most_frequent_value = np.argmax(value_counts)
        vote_mat = unique_values[index_of_most_frequent_value]

        part_mat_pairs[s,ii,:] = np.array([vote_part, vote_mat])

# data = { "part_labels": pred_parts.astype('int16'),
#     "mat_labels": pred_mats.astype('uint8'),
#     "point_grouping": point_grouping.astype('uint8'),
#     "part_mat_pairs": part_mat_pairs.astype('int16'),
#     "shape_preds": pred_objs.astype('uint8'),
#     'out_model_ids': model_ids, 'out_style_ids': style_ids
# }

# with h5py.File('fine_valid_10comp_bpnet_submit.h5', 'w') as f:
#     for name, arr in data.items():
#         f.create_dataset(name, data=arr)
# print('save results done')


In [9]:
from eval_code import *

In [10]:
## read original point clouds

In [11]:
from tqdm import tqdm

hdf_path = '/lustre/scratch/project/k1546/3DCoMPaT-v2/hdf5/'
hdf_path = '/home/lix0i/Xiang/3DCoMPaT/BPNet/hdf5/'

test_data = h5py.File(hdf_path + '{}_{}.hdf5'.format(split, sem_level))
## number of shapes in train/valid/test: 8076, 677, 1256

all_shape_ids = np.array(test_data['shape_id']).astype('str')
all_style_ids = np.array(test_data['style_id']).astype('int32')
print(all_shape_ids.shape)

(12560,)


In [12]:
## read 3d classification results

In [13]:
## number of shapes in train/valid/test: 8076, 677, 1256
use_separate3d = False
if use_separate3d:
    hdf_path = '/lustre/scratch/project/k1546/challenge/class_{}_{}.h5'.format(sem_level, split)
    hdf_path = 'results/class_{}_{}.h5'.format(sem_level, split)
    test_data = h5py.File(hdf_path)
    cls_model_ids = np.array(test_data['out_model_ids']).astype('str')
    cls_style_ids = np.array(test_data['out_style_ids']).astype('int32')
    cls_pred_targets = np.array(test_data['shape_preds']).astype('int32')
    print(cls_pred_targets.shape,)

In [14]:
# reorderring
pred_objs_new = []
new_ord = []

no_matchs = 0
one_matches = 0
two_matches = 0

for i in tqdm(range(len(all_shape_ids))):
    
    model_id = all_shape_ids[i]
    style_id = all_style_ids[i]
    
    idx = np.where((model_ids.astype('str') == model_id) & (style_ids.astype('int') == style_id))[0]
    if len(idx)==0:
        print('no match', model_id, style_id, idx)
        no_matchs += 1
    elif len(idx)==1:
        one_matches += 1
    elif len(idx)>=2:
        print('two match', model_id, style_id, idx)
        two_matches += 1
    
    new_ord.append(idx[0])

    if use_separate3d:
        cls_idx = np.where((cls_model_ids == model_id) & (cls_style_ids == style_id))[0]
        pred_objs_new.append(cls_pred_targets[cls_idx]) # replace with seperate 3d classification results

new_ord = np.array(new_ord)
if use_separate3d:
    pred_objs_new = np.array(pred_objs_new)


100%|██████████| 12560/12560 [00:29<00:00, 429.47it/s]


In [15]:
no_matchs, one_matches, two_matches

(0, 12560, 0)

In [16]:

## save gt for evaluation
data = { "part_labels": gt_parts[new_ord],
    "mat_labels": gt_mats[new_ord],
    "shape_preds": gt_objs[new_ord],
}
with h5py.File('coarse_com50_test_10comp_bpnet_gt.h5', 'w') as f:
    for name, arr in data.items():
        f.create_dataset(name, data=arr)
print('save results done')


save results done


In [17]:
data = { "part_labels": pred_parts[new_ord].astype('int16'),
    "mat_labels": pred_mats[new_ord].astype('uint8'),
    "point_grouping": point_grouping[new_ord].astype('uint8'),
    "part_mat_pairs": part_mat_pairs[new_ord].astype('int16'),
    "shape_preds": pred_objs[new_ord].astype('uint8'),
    'out_model_ids': model_ids[new_ord], 'out_style_ids': style_ids[new_ord]
}

with h5py.File('coarse_com50_test_10comp_bpnet_submit.h5', 'w') as f:
    for name, arr in data.items():
        f.create_dataset(name, data=arr)
print('save results done')

save results done


In [18]:
# data = { "part_labels": pred_parts[new_ord].astype('int16'),
#     "mat_labels": pred_mats[new_ord].astype('uint8'),
#     "point_grouping": point_grouping[new_ord].astype('uint8'),
#     "part_mat_pairs": part_mat_pairs[new_ord].astype('int16'),
#     "shape_preds": pred_objs_new.astype('uint8'),
#     'out_model_ids': model_ids[new_ord], 'out_style_ids': style_ids[new_ord]
# }

# with h5py.File('coarse_com5_test_10comp_bpnet_submit_separate3d.h5', 'w') as f:
#     for name, arr in data.items():
#         f.create_dataset(name, data=arr)
# print('save results done')

In [19]:
# test_annotation_file = 'results/coarse_valid_10comp_bpnet_gt.h5'
# user_submission_file = 'results/coarse_valid_10comp_bpnet_submit.h5'
# evaluate(test_annotation_file, user_submission_file, phase_codename='fdfd')

In [20]:
# test_annotation_file = 'results/coarse2_valid_10comp_bpnet_gt.h5'
# user_submission_file = 'results/coarse2_valid_10comp_bpnet_submit_reordered.h5'
# evaluate(test_annotation_file, user_submission_file, phase_codename='fdfd')

In [21]:
test_annotation_file = 'coarse_com50_test_10comp_bpnet_gt.h5'
user_submission_file = 'coarse_com50_test_10comp_bpnet_submit.h5'
# user_submission_file = 'coarse_com5_test_10comp_bpnet_submit_separate3d.h5'
evaluate(test_annotation_file, user_submission_file, phase_codename='com5_test')

Starting Evaluation.....
loaded user submission
Data Loaded
METRICS EVALUATED
Completed evaluation for Test Phase


{'result': [{'test_split': {'Shape accuracy': 3.7977707006369426,
    'Value': 0.02388535031847134,
    'Value-ALL': 0.0,
    'Value-GRND': 0.0,
    'Value-ALL-GRND': 0.0}}],
 'submission_result': {'test_split': {'Shape accuracy': 3.7977707006369426,
   'Value': 0.02388535031847134,
   'Value-ALL': 0.0,
   'Value-GRND': 0.0,
   'Value-ALL-GRND': 0.0}}}

In [22]:
# test_annotation_file = 'results/coarse2_valid_10comp_bpnet_gt.h5'
# user_submission_file = 'results/coarse2_valid_10comp_bpnet_submit_separate3d.h5'
# evaluate(test_annotation_file, user_submission_file, phase_codename='fdfd')